# CodeGPT - Fine-tuning

In [1]:
!pip install tokenizers
!pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install wandb

     |████████████████████████████████| 3.3MB 5.9MB/s 
     |████████████████████████████████| 194kB 5.4MB/s 
     |████████████████████████████████| 245kB 8.0MB/s 
     |████████████████████████████████| 112kB 7.0MB/s 
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-uwzpjc2h
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-uwzpjc2h
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 870kB 6.0MB/s 
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2106601 sha256=065dc53a329ec8f388f7b676cdcf6436eab43a79409af1704a8ea14719a2bed9
  Stored in directory: /tmp/pip-ephem-wheel-cache-86pak9ps/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=8

In [2]:
cd drive/MyDrive/IFT6010/codexlm/

/content/drive/MyDrive/IFT6010/codexlm


In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!rm -rf runs
!rm -rf wandb

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version "nightly"

In [4]:
import os
os.environ['WANDB_PROJECT'] = 'codexlm'
%env WANDB_PROJECT = codexlm
%env WANDB_LOG_MODEL = true 
%env TOKENIZERS_PARALLELISM=false

env: WANDB_PROJECT=codexlm
env: WANDB_LOG_MODEL=true
env: TOKENIZERS_PARALLELISM=false


# Code Completion

In [ ]:
!python xla_spawn.py --num_cores 8 \
        run_clm.py \
    --model_name_or_path microsoft/CodeGPT-small-java \
    --run_name codegpt-tuned-java-cs-large-flows \
    --output_dir ./model/codegpt-tuned-java-cs-large-flows \
    --train_file ./data/java_train.flow.txt \
    --validation_file ./data/java_valid.flow.txt \
    --block_size 1024 \
    --num_train_epochs 50 \
    --gradient_accumulation_steps 4 \
    --learning_rate=3e-5 \
    --lr_scheduler_type=linear \
    --weight_decay=0.01 \
    --evaluation_strategy epoch \
    --logging_strategy epoch \
    --save_strategy=epoch \
    --load_best_model_at_end True \
    --do_train \
    --do_eval \
    --per_device_train_batch_size 4 \
    --report_to wandb

In [ ]:
!python eval_lm.py \
  --model_name_or_path ./model/codegpt-tuned-java-cs-flows/checkpoint-180/ \
  --eval_path ./data/completion-token/java/test.java-cs.txt.java.flow \
  --code_flows_path ./data/completion-token/java/test.java-cs.txt.java.flow

# Machine Translation

In [ ]:
!python ./code-to-code-trans/code/run.py \
  --model_name_or_path microsoft/codebert-base \
  --model_type roberta \
  --tokenizer_name roberta-base \
  --output_dir ./model/codebert-tuned-java-cs-flows-mt \
  --train_filename  ./data/completion-token/java/train.java-cs.txt.java.flow,./data/completion-token/cs/train.java-cs.txt.cs.flow \
  --dev_filename ./data/completion-token/java/valid.java-cs.txt.java.flow,./data/completion-token/cs/valid.java-cs.txt.cs.flow \
  --max_source_length 512 \
  --max_target_length 512 \
  --beam_size 5 \
  --train_batch_size 4 \
  --eval_batch_size 4 \
  --learning_rate 5e-5 \
  --train_steps 20000 \
  --eval_steps 1000 \
  --do_train \
  --do_eval

2021-04-13 15:30:55.889920: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
04/13/2021 15:30:57 - INFO - __main__ -   Namespace(adam_epsilon=1e-08, beam_size=5, config_name='', dev_filename='./data/completion-token/java/valid.java-cs.txt.java.flow,./data/completion-token/cs/valid.java-cs.txt.cs.flow', do_eval=True, do_lower_case=False, do_test=False, do_train=True, eval_batch_size=4, eval_steps=1000, gradient_accumulation_steps=1, learning_rate=5e-05, load_model_path=None, local_rank=-1, max_grad_norm=1.0, max_source_length=512, max_steps=-1, max_target_length=512, model_name_or_path='microsoft/codebert-base', model_type='roberta', no_cuda=False, num_train_epochs=3.0, output_dir='./model/codebert-tuned-java-cs-flows-mt', seed=42, test_filename=None, tokenizer_name='roberta-base', train_batch_size=4, train_filename='./data/completion-token/java/train.java-cs.txt.java.flow,./data/completion-token/cs/train.java-cs.txt.c